## Tutorial 2: Instatiating a *scenario*

In this tutorial, we will cover the following items:

1. Create *actors*, *activities*, *events*, and *static physical things*
2. Instantiate a *scenario*
3. Show all tags of the *scenario*
4. Use the `comprises` function of a *scenario category*
5. Export the objects

In [ ]:
# Before starting, let us do the necessary imports
from domain_model import Activity, ActivityCategory, Actor, ActorCategory, Constant, Scenario, \
    Sinusoidal, State, StateVariable, Tag, VehicleType

## 1. Create *actors*, *activities*, *events*, and *static physical things*

In this tutorial, we will create a *scenario* in which another vehicle changes lane such that it becomes the ego vehicle's leading vehicle. This is the quantitative counterpart of the *scenario category* that we created in the [previous tutorial](./Tutorial%201%20Scenario%20category.ipynb). The *scenario* is depicted in the figure below. Here, the blue car represents the ego vehicle and the red car represents the vehicle that performs the cut in.

<img src="./examples/images/cut-in.png" alt="Cut in" width="400"/>

Just as with a *scenario category*, we first need to create the components of a *scenario*. To create the *scenario*, we first need to create the *actors*, *activities*, *events*, and *static physical things*. Also, *dynamic physical things* could be part of a *scenario*, but for this tutorial, we do not consider *dynamic physical things* (other than the *actors*, which, technically speaking, are *dynamic physical things*). As we have already seen in the [previous tutorial](./Tutorial%201%20Scenario%20category.ipynb), the objects have a `name`, a `uid` (a unique ID), and `tags`. Additionally, these quantitative building blocks contain a reference to their qualitative counterparts.

### 1.1 Create *actors*

Now let us create the *actors*. We need the qualitative counterparts, the *actor categories*, and we will use the *actor categories* from the [previous tutorial](./Tutorial%201%20Scenario%20category.ipynb). Note that we can ignore the `uid` for now.

In [ ]:
# First create the actor categories
EGO_VEHICLE_QUALITATIVE = ActorCategory(VehicleType.Vehicle, name="Ego vehicle", 
                                        tags=[Tag.EgoVehicle, Tag.RoadUserType_Vehicle])
TARGET_VEHICLE_QUALITATIVE = ActorCategory(VehicleType.Vehicle, name="Target vehicle",
                                           tags=[Tag.RoadUserType_Vehicle])

To create the actors, we can define the initial state. Because we will later quantify the speed of both actors as well as the lateral position using the activities, we will not define this now. However, the longitudinal position will not be defined by the activities, so we will use the `initial_states` to specify the initial longitudinal position. The ego vehicle starts at $x=0$ m and the target vehicle starts at $x=70$ m.

For actors, the objective can be defined using the `desired_states`. For now, we will set the target for both vehicles at $x=400$ m and $y=0$ m. The ego vehicle's objective is to maintain the speed of $v=28$ m/s, while the target vehicle's objective is to drive at a speed of $v=22$ m/s. 

In [ ]:
# Now we can create the actors.
EGO_VEHICLE = Actor(EGO_VEHICLE_QUALITATIVE, 
                    initial_states=[State(StateVariable.LONGITUDINAL_POSITION, 0)],
                    desired_states=[State(StateVariable.LONGITUDINAL_POSITION, 400),
                                    State(StateVariable.LATERAL_POSITION, 0),
                                    State(StateVariable.SPEED, 28)],
                    properties=dict(color="blue"),
                    name="ego vehicle")
TARGET_VEHICLE = Actor(TARGET_VEHICLE_QUALITATIVE,
                       initial_states=[State(StateVariable.LONGITUDINAL_POSITION, 70)],
                       desired_states=[State(StateVariable.LONGITUDINAL_POSITION, 400),
                                       State(StateVariable.LATERAL_POSITION, 0),
                                       State(StateVariable.SPEED, 22)],
                       properties=dict(color="red"),
                       name="target vehicle")

### 1.2 Create *activities* and *events*

As with the actors, we first need to create the *activity categories*. We will create *activity categories* for the following type of activities:

- Cruising
- Decelerating
- Accelerating
- Following lane
- Changing lane

In [ ]:
# First create the activity categories
CRUISING_CATEGORY = ActivityCategory(Constant(), StateVariable.SPEED, name="Cruising",
                                     description="Cruising at a constant positive speed",
                                     tags=[Tag.VehicleLongitudinalActivity_DrivingForward_Cruising])
BRAKING_CATEGORY = ActivityCategory(Sinusoidal(), StateVariable.SPEED, name="Braking",
                                    description="Decelerating to a lower speed",
                                    tags=[Tag.VehicleLongitudinalActivity_DrivingForward_Braking])

In [ ]:
CRUISING_EGO = Activity(CRUISING_CATEGORY, dict(xstart=28), start=0, end=5, name="Initial cruising ego")
BRAKING_EGO = Activity(BRAKING_CATEGORY, dict(xstart=28, xend=20), start=CRUISING_EGO.end, end=10,
                       name="Braking ego")